# Dataset repacking implementation

Advance dataset operations, of sorting, offset, and length support

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-datapack"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [3]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 6 --n_embd 2048 \
        --vocab_size world --skip-if-exists \
        "../model/L6-D2048-world-v5base-init.pth"

[2024-01-28 02:00:22,765] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
---- Initializing model ----
No of layers: 6
Embedding size: 2048
Output model path: ../model/L6-D2048-world-v5base-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


# Build the datapack

In [60]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 datapack_build.py "{NOTEBOOK_DIR}/config/datapack-build.yaml"

>> Starting datapack build process for: /home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/datapack-build.yaml
>> Preparing dataset - index:  0
Map (num_proc=160): 100%|██████████| 1258/1258 [00:00<00:00, 2062.82 examples/s]
num_proc must be <= 13. Reducing num_proc to 13 for dataset of size 13.
Map (num_proc=13): 100%|█████████████████| 13/13 [00:00<00:00, 59.14 examples/s]
Saving the dataset (1/1 shards): 100%|█| 1258/1258 [00:00<00:00, 18965.31 exampl
Saving the dataset (1/1 shards): 100%|█| 13/13 [00:00<00:00, 2478.00 examples/s]
>> Preparing dataset - index:  1
Map (num_proc=160): 100%|██████████| 2429/2429 [00:02<00:00, 1065.18 examples/s]
Saving the dataset (1/1 shards): 100%|█| 18199/18199 [00:00<00:00, 21397.84 exam
Saving the dataset (1/1 shards): 100%|█| 2429/2429 [00:00<00:00, 73285.75 exampl
>> Total approx train batches ( full | random ) : 9  (  7  |  2  )
Map (num_proc=160): 100%|████████| 18432/18432 [00:03<00:00, 5626.16 examples/s]
>> Final datas

# Test train

In [63]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="disabled" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/datapack-train.yaml" \
        --model.load_model="../model/L6-D2048-world-v5base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/datapack-validaiton-train/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Multi Datapack Validation - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=8 \
        --trainer.fast_dev_run=2 \
        --trainer.devices="{GPU_DEVICES}"

[2024-01-28 06:58:53,174] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/datapack-train.yaml', '--model.load_model=../model/L6-D2048-world-v5base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/datapack-validaiton-train/', '--trainer.logger.init_args.name=infctx-v5-datapack - Multi Datapack Validation - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=8', '--trainer.fast_dev_run=2', '--trainer.devices=auto'], args=['f

# Actual train

In [64]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/datapack-train.yaml" \
        --model.load_model="../model/L6-D2048-world-v5base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/datapack-validaiton-train/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Multi Datapack Validation - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=8 \
        --trainer.devices="{GPU_DEVICES}"

[2024-01-28 06:59:29,981] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/test-dataset-repack.yaml', '--model.load_model=../model/L6-D2048-world-v5base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/datapack-validaiton-train/', '--trainer.logger.init_args.name=infctx-v5-datapack - Multi Datapack Validation - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=8', '--trainer.devices=auto'], args=['fit', '-c', '/home/recur